In [5]:
# %load_ext autotime

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

In [7]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Load Usage data
with open('App_usage_trace.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split()
        df.append(data)
usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")) # Convert to datetime object
usage['loc'] = usage['loc'].astype('int64')
usage["date"] = usage["timestamp"].dt.strftime("%m-%d")
usage.head()

KeyboardInterrupt: 

# Identify users that have less than X records per day, see what is the impact of the removal

In [6]:
# group uid and date, purpose is to see the number of request that a user made for each day
user_usage_perday = usage.groupby(["uid", "date"]).size().reset_index()
# rename columns for easy slicing
user_usage_perday.columns = ["uid", "date", "count"]
# number of users that does not have X counts per day
# change this
required_count = 10

# slice dataset to only remain those rows that has less than 20 count
df_less10day = user_usage_perday[user_usage_perday["count"] < required_count]
# group dataset with uid to get the count of the occurance of the uid, so we can determine which users have just 1 day or more
uid_less10day = df_less10day.groupby(["uid"]).size().reset_index()
# rename columns for easy slicing
uid_less10day.columns = ["uid", "count"]
print("="*100)
print("Current condition: remove user that has less than {} records per day".format(required_count))
print("="*100)
print("There are {} users that does not meet the requirement". format(uid_less10day.shape[0]))
print("-"*100)
print("There are {} users that have less than {} records for exact 1 day".format((uid_less10day.shape[0]- uid_less10day[uid_less10day["count"] != 1].shape[0]), required_count))
print("-"*100)
print("There are {} users that have less than {} records for more than 1 day".format(uid_less10day[uid_less10day["count"] != 1].shape[0], required_count))
print("-"*100)
print("Under current condition, {} rows will be removed, {:.5%} of the entire dataset ".format(df_less10day["count"].sum(), df_less10day["count"].sum()/414171950))
print("="*100)

Current conditions: remove user that has less than 10 records per day
There are 200 users that does not meet the requirement
----------------------------------------------------------------------------------------------------
There are 140 users that have less than 10 records for exact 1 day
----------------------------------------------------------------------------------------------------
There are 60 users that have less than 10 records for more than 1 day
----------------------------------------------------------------------------------------------------
Under current condition, 1263 rows will be removed, 0.00030% of the entire dataset 
time: 641 ms


# Implementation of the cleaning

In [10]:
required_count = 10
# slice dataset to only remain those rows that has less than 20 count
user_usage_perday = usage.groupby(["uid", "date"]).size().reset_index()
user_usage_perday.columns = ["uid", "date", "count"]

df_less10day = user_usage_perday[user_usage_perday["count"] < required_count]
records_remove = np.array(df_less10day)
usage_copy = usage.copy()
print("Before removal the dataset has {} rows".format(usage.shape[0]))
for i in range(len(records_remove)):
    uid = records_remove[i][0]
    date = records_remove[i][1]
    usage_copy = usage_copy.drop(usage_copy[(usage_copy["uid"] == uid) & (usage_copy["date"] == date)].index)
print("After removal the dataset has {} rows, {} rows has been removed".format(usage_copy.shape[0], (usage.shape[0] - usage_copy.shape[0])))

Before removal the dataset has 4171950 rows
After removal the dataset has 4170687 rows, 1263 rows has been removed
time: 4min 56s


In [11]:
# rename the cleaned dataset 
usage_cleaned = usage_copy.copy()

time: 125 ms


In [ ]:
usage_cleaned

# Adpot Oscillation

In [96]:
# Function to align the Origin and Destination (O/D)
def sort_movement_tuples(jump):
    return sorted(list(jump))

# Define a function that can output the frequency pair of a user's mobility pattern
def show_frequent_pair(uid=888, threshold=5, jump_occurence=10):
    # Nathan: replaced usage with usage_cleaned
    user_uid = usage_cleaned[usage_cleaned['uid'] == str(uid)]

    # Set the thresold of time gap allowed between movement
    delta_threshold = timedelta(seconds=threshold)

    # Create the columns of the next time and lcoation
    user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
    user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)

    # Keep only movements
    user_uid = user_uid[user_uid['loc'] != user_uid['next_loc']]

    # Get the time gaps between movements
    user_uid['time_delta'] = user_uid['next_timestamp'] - user_uid['timestamp']

    # Get teleports and count the teleports
    # Should be able to identify the return trips
    teleports = user_uid[user_uid['time_delta'] < delta_threshold]
    teleports['tele'] = list(zip(teleports['loc'], teleports['next_loc']))
    # print(teleports['tele'].value_counts())

    # # Group and count the same jump
    tele_counts = teleports['tele'].value_counts().reset_index()
    tele_counts.columns = ['jump', 'count']

    # Align the O/D of the jump to identify the pairs
    tele_counts['jump'] = tele_counts['jump'].apply(lambda i: sort_movement_tuples(i))

    # Get the jumps that has a reasonable number of occurence
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]
    tele_counts['jump'] = tuple(tele_counts['jump'])

    # print the record for reference
    # Nathan: added reset_index()
    frequent_pairs = tele_counts.groupby(['jump']).sum().reset_index()

    return frequent_pairs

time: 0 ns


In [87]:
# Function to align the Origin and Destination (O/D)
def sort_movement_tuples(jump):
    return sorted(list(jump))

# show the period where the oscillation happens
def present_oscillation(uid=888, threshold=5, jump_occurence=10):
    # Nathan: replaced usage with usage_cleaned
    user_uid = usage_cleaned[usage_cleaned['uid'] == str(uid)]

    # Set the thresold of time gap allowed between movement
    delta_threshold = timedelta(seconds=threshold)

    # Create the columns of the next time and lcoation
    user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
    user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)

    # Keep only movements
    user_uid = user_uid[user_uid['loc'] != user_uid['next_loc']]

    # Get the time gaps between movements
    user_uid['time_delta'] = user_uid['next_timestamp'] - user_uid['timestamp']

    # Get teleports and count the teleports
    # Should be able to identify the return trips
    teleports = user_uid[user_uid['time_delta'] < delta_threshold]
    teleports['tele'] = list(zip(teleports['loc'], teleports['next_loc']))
    # teleports['tele'] = teleports['tele'].apply(lambda i: sort_movement_tuples(i))
    # print(teleports['tele'].value_counts())

    # # Group and count the same jump
    tele_counts = teleports['tele'].value_counts().reset_index()
    tele_counts.columns = ['jump', 'count']
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]


    # # Align the O/D of the jump to identify the pairs
    # tele_counts['jump'] = tele_counts['jump'].apply(lambda i: sort_movement_tuples(i))

    # Get the jumps that has a reasonable number of occurence
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]
    # tele_counts['jump'] = tuple(tele_counts['jump'])

    #return the rows that has tele_count in the "tele" column, so it returns the rows where the oscillation happens
    return teleports[teleports['tele'].isin(tele_counts["jump"])]


time: 0 ns


In [102]:
# show the frequent pair
show_frequent_pair(0,10,5)

<ipython-input-96-20c6b7ee343d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-96-20c6b7ee343d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-96-20c6b7ee343d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

,jump,count
0,"(1361, 2304)",14


time: 6.23 s


In [104]:
# show the oscillation period
present_oscillation(0,10,5)

<ipython-input-87-deb54b23e21a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-87-deb54b23e21a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-87-deb54b23e21a>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

,uid,timestamp,loc,app_id,traffic,date,next_loc,next_timestamp,time_delta,tele
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22,2304,2016-04-22 14:26:31,00:00:00,"(1361, 2304)"
1789,0,2016-04-22 14:26:41,2304,361,0.003,04-22,1361,2016-04-22 14:26:44,00:00:03,"(2304, 1361)"
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22,2304,2016-04-22 14:26:45,00:00:00,"(1361, 2304)"
1794,0,2016-04-22 14:26:46,2304,41,0.007,04-22,1361,2016-04-22 14:26:48,00:00:02,"(2304, 1361)"
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22,2304,2016-04-22 14:26:48,00:00:00,"(1361, 2304)"
1796,0,2016-04-22 14:26:48,2304,361,0.002,04-22,1361,2016-04-22 14:26:49,00:00:01,"(2304, 1361)"
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22,2304,2016-04-22 14:26:54,00:00:05,"(1361, 2304)"
1798,0,2016-04-22 14:26:54,2304,361,0.006,04-22,1361,2016-04-22 14:26:55,00:00:01,"(2304, 1361)"
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22,2304,2016-04-22 14:26:55,00:00:00,"(1361, 2304)"
1800,0,2016-04-22 14:26:55,2304,361,0.003,04-22,1361,2016-04-22 14:26:56,00:00:01,"(2304, 1361)"


time: 5 s


# Ignore the following cells


# Replace the loc id with less count with the loc id with higher count in the frequent pair

In [74]:
# # Show the frequent pairs of user 13, gap less than 3 seconds, and occurence higher than 5 times (both directions count)
freq_pair = np.array(show_frequent_pair(0,10,5))
freq_pair

<ipython-input-72-08347122befa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-72-08347122befa>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-72-08347122befa>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

array([[(2304, 1361), 7],
       [(1361, 2304), 7]], dtype=object)

time: 6.12 s


In [75]:
print(freq_pair[0][0])
print(freq_pair[1][0])

(2304, 1361)
(1361, 2304)
time: 0 ns


In [18]:
df_exp = usage_cleaned[(usage_cleaned["loc"] == freq_pair[0][0][0]) & (usage_cleaned["uid"] == str(0))]
df_exp

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22
1801,0,2016-04-22 14:26:56,1361,41,0.001,04-22
1803,0,2016-04-22 14:26:57,1361,41,0.003,04-22
1805,0,2016-04-22 14:27:00,1361,361,0.003,04-22


time: 281 ms


In [19]:
df_exp1 = usage_cleaned[(usage_cleaned["loc"] == freq_pair[0][0][1]) & (usage_cleaned["uid"] == str(0))]
df_exp1

,uid,timestamp,loc,app_id,traffic,date
1788,0,2016-04-22 14:26:31,2304,361,0.003,04-22
1789,0,2016-04-22 14:26:41,2304,361,0.003,04-22
1793,0,2016-04-22 14:26:45,2304,41,0.003,04-22
1794,0,2016-04-22 14:26:46,2304,41,0.007,04-22
1796,0,2016-04-22 14:26:48,2304,361,0.002,04-22
1798,0,2016-04-22 14:26:54,2304,361,0.006,04-22
1800,0,2016-04-22 14:26:55,2304,361,0.003,04-22
1802,0,2016-04-22 14:26:56,2304,361,0.002,04-22
1804,0,2016-04-22 14:26:57,2304,41,0.001,04-22


time: 297 ms


In [146]:
df_exp = pd.concat([df_exp, df_exp1])
df_exp

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22
1801,0,2016-04-22 14:26:56,1361,41,0.001,04-22
1803,0,2016-04-22 14:26:57,1361,41,0.003,04-22
1805,0,2016-04-22 14:27:00,1361,361,0.003,04-22


time: 16 ms


In [150]:
if (df_exp[(df_exp["loc"] == freq_pair[0][0][0]) & (df_exp["uid"] == str(0))].shape[0]) > (df_exp[(df_exp["loc"] == freq_pair[0][0][1]) & (df_exp["uid"] == str(0))].shape[0]):
    df_exp["loc"] = df_exp["loc"].replace(freq_pair[0][0][1], freq_pair[0][0][0])
df_exp

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22
1801,0,2016-04-22 14:26:56,1361,41,0.001,04-22
1803,0,2016-04-22 14:26:57,1361,41,0.003,04-22
1805,0,2016-04-22 14:27:00,1361,361,0.003,04-22


time: 15 ms
